In [319]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [320]:
import tensorflow_hub as hub
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

In [321]:
train_df = pd.read_csv('./preprocessing/train.csv')
test_df = pd.read_csv('./preprocessing/test.csv')

In [322]:
module_url = "https://tfhub.dev/google/nnlm-en-dim128/2"
embed = hub.KerasLayer(module_url)
embeddings = embed(["A long sentence.", "single-word",
                  "http://example.com"])
print(embeddings.shape)  #(3,128)

(3, 128)


In [323]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/3")

In [324]:
X_train_embeddings = embed(train_df.text.values)
X_test_embeddings = embed(test_df.text.values)
ans = pd.read_csv('./dataset/ans.csv')['target'].values

In [325]:
X = np.array(X_train_embeddings['outputs'])
y = train_df.target

In [326]:
from sklearn.model_selection import train_test_split
# Test train split 
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [332]:
params = {
    'learning_rate': 0.04,
    'n_estimators': 1500,
    'colsample_bytree': 0.4,
    'metric':'auc'
}

In [333]:
text_clf = LGBMClassifier(**params)

In [334]:
text_clf.fit(X_train, y_train,
             eval_set=[(X_train, y_train),
                       (X_val, y_val)],
             verbose=200, early_stopping_rounds=20,
            )

LGBMClassifier(colsample_bytree=0.4, learning_rate=0.04, metric='auc',
               n_estimators=1500)

In [335]:
text_clf.fit(X_train, y_train)
Y_pred_val = text_clf.predict(X_val)
accuracy_score(y_pred= Y_pred_val, y_true= y_val)

0.8194353250164149

In [336]:
# text_clf.fit(X_train_embeddings['outputs'], train_df.target.values)
pred_test = text_clf.predict(X_test_embeddings['outputs'])
ans = pd.read_csv('./dataset/ans.csv')['target'].values
accuracy_score(y_pred= pred_test, y_true= ans)

0.816426601287159